# Start 
# Create Point Clouds to Calclulate Femur Head Centers

<div class="alert alert-block alert-info">
<b>In3dSlicer GUI</b>:
<p>1. Load MRI last folder</p>
<p>2. Load markers template. Save project and files in a new folder inside the individual, make sure to change the default folder for all files.</p>
<p>3. In markup module, create two point lists: <emp>femur_head_right</emp> and <emp>femur_head_left</emp>.</p>

<p>4. For each, left and right accordingly, collect lms on femur heads circular outline, red and green slices: move slices to the level of the largest femur head radius (approximately near fovea).</p>

<p>5. Create a new empty markups ladmark list 'lms'.</p>

<p>Once finished, follow instructions below: either or execute code or interact with <b>GUI</b>.</p>
</div>

# Load libraries and initiate functions

In [2]:
import numpy as np
import slicer
import math
import vtk

In [3]:
# helper functions:

def sphereFit(data):

    #   Assemble the A matrix
    spX = np.array(data[:,0])
    spY = np.array(data[:,1])
    spZ = np.array(data[:,2])
    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A,f, rcond = None)

    #   solve for the radius
    t = np.dot(C[0],C[0])+np.dot(C[1],C[1])+np.dot(C[2],C[2])+C[3]
 
    radius = math.sqrt(t[0])
    center = np.array([C[0], C[1], C[2]])

    return radius, center

def setSlicePoseFromSliceNormalAndPosition(sliceNode, sliceNormal, slicePosition, defaultViewUpDirection=None, backupViewRightDirection=None):
    
    ## Fix up input directions
    if defaultViewUpDirection is None:
        defaultViewUpDirection = [0,0,1]
    if backupViewRightDirection is None:
        backupViewRightDirection = [-1,0,0]
    if sliceNormal[1]>=0:
        sliceNormalStandardized = sliceNormal
    else:
        sliceNormalStandardized = [-sliceNormal[0], -sliceNormal[1], -sliceNormal[2]]

        ## Compute slice axes
    sliceNormalViewUpAngle = vtk.vtkMath.AngleBetweenVectors(sliceNormalStandardized, defaultViewUpDirection)
    angleTooSmallThresholdRad = 0.25 # about 15 degrees
    if sliceNormalViewUpAngle > angleTooSmallThresholdRad and sliceNormalViewUpAngle < vtk.vtkMath.Pi() - angleTooSmallThresholdRad:
        viewUpDirection = defaultViewUpDirection
        sliceAxisY = viewUpDirection
        sliceAxisX = [0, 0, 0]
        vtk.vtkMath.Cross(sliceAxisY, sliceNormalStandardized, sliceAxisX)
    else:
        sliceAxisX = backupViewRightDirection
    
    ## Set slice axes
    sliceNode.SetSliceToRASByNTP(sliceNormalStandardized[0], sliceNormalStandardized[1], sliceNormalStandardized[2],
        sliceAxisX[0], sliceAxisX[1], sliceAxisX[2],
        slicePosition[0], slicePosition[1], slicePosition[2], 0)

# Collect Key Starting Landmarks 

In [ ]:
# left femur
left_fh = slicer.util.getNode('femur_head_left')
left_fh = slicer.util.arrayFromMarkupsControlPoints(left_fh)
left_radius, left_center = sphereFit(left_fh)
left_center

In [7]:
# right femur
right_fh = slicer.util.getNode('femur_head_right')
right_fh = slicer.util.arrayFromMarkupsControlPoints(right_fh)
right_radius, right_center = sphereFit(right_fh)
right_radius

21.431554854401803

In [8]:
# Import lms and add center points to it
lms = slicer.util.getNode('lms')
left_center_point = lms.AddControlPoint(left_center)
lms.SetNthControlPointLabel(left_center_point, 'femur_l_center')
right_center_point = lms.AddControlPoint(right_center)
lms.SetNthControlPointLabel(right_center_point, 'femur_r_center')

C:/Users/EKATAR~1/AppData/Local/Temp/xpython_18636/379390962.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  left_center_point = lms.AddControlPoint(left_center)
C:/Users/EKATAR~1/AppData/Local/Temp/xpython_18636/379390962.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  right_center_point = lms.AddControlPoint(right_center)


<div class="alert alert-block alert-info">
    <b>In 3dSlicer GUI</b>, place the following key markers:
    
 1) _torso_origin_in_pelvis_ : defined as a point at the posterior edge of the intervertebral disk between L5 and sacrum on the yellow mid-sagittal slice. This point must later be in the same slice as _pelvis_origin_ and _pelvis_origin_in_ground_ .
   
 3) _ASIS_l_ and _ASIS_r_ : defined as the anteriormost point of the iliac blades that touches skin on the red slice; on the green slice, the same point should look like as small round shadow and on the yellow slice it is anteroinferior point of the iliac blade.
    
 4) _PSIS_l and _PSIS_r_ : defined as posterior and iferior points of iliac blades. They should be positioned posterior to sacrum. On the red slice, the sacrum must still be visible, probably it's uppermost vertebra.
    
 5) _pub_infer_c_ and _pub_super_c_ : defined as the inferior and superior point on the sagittal optline of the pubic symphysis. Please note that the two points should lie along the longest axis of the sympysis.
    
</div>

In [18]:
# re-load lms node
lms = slicer.util.getNode('lms')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

asis_l = lms_list[lms.GetControlPointIndexByLabel('ASIS_l')]
asis_r = lms_list[lms.GetControlPointIndexByLabel('ASIS_r')]

pelvis_origin = np.mean((asis_l, asis_r), axis = 0)
pelvis_origin_point = lms.AddControlPoint(vtk.vtkVector3d(pelvis_origin))
lms.SetNthControlPointLabel(pelvis_origin_point, 'pelvis_origin')

psis_l = lms_list[lms.GetControlPointIndexByLabel('PSIS_l')]
psis_r = lms_list[lms.GetControlPointIndexByLabel('PSIS_r')]
psis_c = np.mean((psis_l, psis_r), axis = 0)

pub_infer_c = lms_list[lms.GetControlPointIndexByLabel('pub_infer_c')]
pub_super_c = lms_list[lms.GetControlPointIndexByLabel('pub_super_c')]

In [4]:
# define new normal for the red slice
red_slice_new_normal = np.cross((pelvis_origin - psis_l), (pelvis_origin - psis_r))
red_slice_new_normal = red_slice_new_normal/np.linalg.norm(red_slice_new_normal)

In [6]:
# reorient red slice
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(
    redSliceNode, red_slice_new_normal, 
    np.mean((psis_l, psis_r, pelvis_origin), 
            axis = 0)) ## red slice now shows PSIS_l, PSIS_r and pelvis_origin points

In [7]:
# reorient green slice
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(
    greenSliceNode, np.array([0,-1,0]), 
    np.mean((psis_l, psis_r, pelvis_origin), 
            axis = 0)) ## green slice now crossects pelvis in the middle

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, collect a number of key markers:</b>
    
1)  _ilium_l_ and _ilium_r_ : move the red slice up so that it touches the tops of iliac crests. The green slice position marks the centre between psis and pelvis origin markers. The red slice should show an elongated shadow of the left or/and right iliac crests are visible but do not quite reach the green line, which marks the position of the green slice when observed on the red. The points should be placed at the anterior corner of the right or left iliac crest shadow. Once done, return the red slice to default poisiton (see panel above the slice, change 'reformat' to 'axial'). 

2) _knee_l_lat_ and _knee_r_lat_ : these shoule be placed on the lateralmost points of lateral femoral epicondyles.

3) _ankle_l_lat_ and _ankle_l_lat_ : these should be placed on lateralmost points of fibulas# lateral malleolae.
    
   </div>

In [27]:
# reorient yellow slice to be perpendicular ASIS axis:
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
asis_axis = (asis_r - asis_l)/np.linalg.norm(asis_r - asis_l)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, asis_axis , pub_super_c)

In [23]:
# reorient green slice again so that it is perpendicular to the pubic symphyis and passess through pub_sup_c:
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
psis_symph_axis = (pub_super_c - psis_c)/np.linalg.norm(pub_super_c - psis_c)
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, psis_symph_axis , pub_super_c)

In [20]:
# reorient red slice again so that it is perpendicular to the pubic symphyis and passess through pub_sup_c:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
symph_axis = (pub_super_c - pub_infer_c)/np.linalg.norm(pub_super_c - pub_infer_c)
setSlicePoseFromSliceNormalAndPosition(redSliceNode, symph_axis, pub_super_c)

<div class="alert alert-block alert-info">pubic tubercle points should lie on the line of the red/green crossection at the point of the pub_super_c</div>

In [29]:
# reorient red slice to make it anterior as in Rajagopal et al 2016 passessing through pub_sup_c:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
pelvis_origin_to_symph = (pub_super_c - pelvis_origin)/np.linalg.norm(pub_super_c - pelvis_origin)
fronal_norm = np.cross(asis_axis, pelvis_origin_to_symph)

setSlicePoseFromSliceNormalAndPosition(redSliceNode, fronal_norm, pub_super_c)

# Warp the template to the space of the MRI

<div class="alert alert-block alert-info">
<b> Create a new lm markup, name it any way you like, and do the following:</b>
    
1) From _orientation_template_, copy points that match those already in lms to the new lm markup; 
2) Change the list position of the points so that your new markup is identical to lms;
3) Use module 'IGT' -> 'fiducial registration' to create new transform from your new markup to lms (warping); 
4) Move the _orientation_template_ landmarks to the transform dependancy to check points fit the image; 
5) If the above answer is 'yes', return _orientation_template_ to the independent position, clone it and rename the clone to 'orientation'.
6) Move the 'orientation' clone to the transform dependentcy and harden the transform.

Superimposed landmarks should fit exactly.
</div>

# Adjust the position of warped landmarks

<div class="alert alert-block alert-info">
<b>Adjust location of projected landmarks, starting from the top:</b>

_gr_troch_as_l_ and _gr_troch_as_r_** -- greater trochanter anterosuperior -- defined as the anterior point on the smallest red crossection through the greater trochanter, roughly corresponds with the superior point of the greater trochanter anatomically; one needs to be careful to distinguish between the bone, fat and tendons

_gr_troch_lat_l_ and _gr_troch_lat_r_** -- greater trochanter lateral -- defined as the lateralmost point on the green slice through greater trochanter, it should lie posteriorly to the gr_troch_as point

**__NB position of trochanter points is likely to be influenced by the rotaion of the femur: watch the effects__

_isch_tuber_l_ and _isch_tuber_r_ -- ischial tuberosity -- defined as the posteriormost landmark on ischial tuberosity, on the green slice is marked as black endpoint of the bone

_pub_infer_c_ -- pubis inferior central -- inferiormost point of the pubic sympysis impriz on the central yellow slice (controlled by the red and green)

_isch_infer_l_ and _isch_infer-r_ -- ischial [bone] inferior -- defined as infero-anteriormost point of the imprint of the ischio-pubic arch on the red slice (control on the green and yellow)

**__Here, one needs to skip midshaft points as their position depends on the axis of the femur and therefore its inclination relative to slices - they will be adjusted later__

_knee_l_med_ and _knee_r_med_ -- knee medial, defined as the medial-most point of femur medial epicondyle on the red slice; this point should be roughly inline with the lateral knee point and the center of the intercondylar fossa

_ankle_l_med_ and _ankle_r_med_ -- medial malleolus on left and right tibias; defined as the most medially prominent point on tibial malleolus
</div>

## Calculate and update central points for the knee and the ankle

In [14]:
## note that knee needs two sets of points updated: the ones that will be in child and 
## the ones that will be in parent frames

lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_lat = lms_list[lms.GetControlPointIndexByLabel('knee_l_lat')]
knee_l_med = lms_list[lms.GetControlPointIndexByLabel('knee_l_med')]
knee_l_center = lms.GetControlPointIndexByLabel('knee_l_center')
knee_l_center_in_femur_l = lms.GetControlPointIndexByLabel('knee_l_center_in_femur_l')
temp1 = np.mean((knee_l_lat, knee_l_med), axis = 0)
lms.SetNthControlPointPosition(knee_l_center, temp1[0], temp1[1], temp1[2])
lms.SetNthControlPointPosition(knee_l_center_in_femur_l, temp1[0], temp1[1], temp1[2])

knee_r_lat = lms_list[lms.GetControlPointIndexByLabel('knee_r_lat')]
knee_r_med = lms_list[lms.GetControlPointIndexByLabel('knee_r_med')]
knee_r_center = lms.GetControlPointIndexByLabel('knee_r_center')
knee_r_center_in_femur_r = lms.GetControlPointIndexByLabel('knee_r_center_in_femur_r')
temp2 = np.mean((knee_r_lat, knee_r_med), axis = 0)
lms.SetNthControlPointPosition(knee_r_center, temp2[0], temp2[1], temp2[2])
lms.SetNthControlPointPosition(knee_r_center_in_femur_r, temp2[0], temp2[1], temp2[2])

ankle_l_lat = lms_list[lms.GetControlPointIndexByLabel('ankle_l_lat')]
ankle_l_med = lms_list[lms.GetControlPointIndexByLabel('ankle_l_med')]
ankle_l_center = lms.GetControlPointIndexByLabel('ankle_l_center')
temp3 = np.mean((ankle_l_lat, ankle_l_med), axis = 0)
lms.SetNthControlPointPosition(ankle_l_center, temp3[0], temp3[1], temp3[2])

ankle_r_lat = lms_list[lms.GetControlPointIndexByLabel('ankle_r_lat')]
ankle_r_med = lms_list[lms.GetControlPointIndexByLabel('ankle_r_med')]
ankle_r_center = lms.GetControlPointIndexByLabel('ankle_r_center')
temp4 = np.mean((ankle_r_lat, ankle_r_med), axis = 0)
lms.SetNthControlPointPosition(ankle_r_center, temp4[0], temp4[1], temp4[2])

## Locate lat and med points on tibias, and update locations of patella landmarks

### Left tibia

In [4]:
# First, calculate tibial axes
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
ankle_l_center = lms_list[lms.GetControlPointIndexByLabel('ankle_l_center')]
tibia_l_normal = (knee_l_center-ankle_l_center)/np.linalg.norm(knee_l_center-ankle_l_center)

In [5]:
# Second, reorient the red slice perpendicular to the ankle-knee axis for the left tibia
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, knee_l_center)

<div class="alert alert-block alert-info">
    <b>In 3dSclicer GUI</b>, move the red slice to intersect with the uppermost part of the tibial plateau, and place the lateral and medial across the longest axis.
        </div>

In [6]:
# import medial and lateral points into jupyter
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

tibia_l_lat = lms_list[lms.GetControlPointIndexByLabel('tibia_l_lat')]
tibia_l_med = lms_list[lms.GetControlPointIndexByLabel('tibia_l_med')]


<div class="alert alert-block alert-info">
<b>While here</b>, adjust the position of landmarks on fibula: <em>fibula_l_as</em> is the anterior tip of the proximal end of the bone; <em>fibula_l_sup</em> - is the superior tip of the proximal head. These are best seen on the yellow slice but can be also controlled on the red slice.
</div>
<div class="alert alert-block alert-info">
    <b>Contunue using the same orientation of the red slice</b>, move the slice up so that it cuts across the widest part of the left patella, and adjust the lateral, medial, and anterior points of the patella in this view. The anterior point should be roughly in the middle of the frontal surface. Then, follow the code below to adjust the orientation of the yellow slice :
    </div>

In [7]:
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

patella_lat_l = lms_list[lms.GetControlPointIndexByLabel('patella_lat_l')]
patella_med_l = lms_list[lms.GetControlPointIndexByLabel('patella_med_l')]
patella_l_anter = lms_list[lms.GetControlPointIndexByLabel('patella_l_anter')]

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
patella_l_normal = (patella_med_l - patella_lat_l)/np.linalg.norm(patella_med_l - patella_lat_l)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, patella_l_normal, patella_l_anter)

<div class="alert alert-block alert-info">
 Then, <b>switch to the yellow slice</b> and display the anterior point of the patella on it. From here, move the slice so that the lowest tip of the patella is visible and adjust the position of <em>patella_l</em> landmark. Move <em>patella_sup_l</em> such that it demarcates the top of the left patella on the same yellow slice as <em>patella_l</em>. 
</div>

In [8]:
# rotate the green slice so that it is parallel to patella mediolateral axis and center it on the ankle:
# the frontal plane should go through the ankle center and through the medial and lateral points on the tibial plateau
tibia_l_frontal_plane_normal = np.cross(tibia_l_normal, patella_l_normal)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")

setSlicePoseFromSliceNormalAndPosition(greenSliceNode, tibia_l_frontal_plane_normal, ankle_l_center)

<div class="alert alert-block alert-info">
 <b>As you look at the green slice, move the location of the red slice </b> so that it cuts across tha tibial epiphysis just below the joint surface. Adjust the position of <em>tibia_l_lat</em> and <em>tibia_l_med</em> so that they are visible on both, green and red slices, i.e to the plane that goes through the ankle centre.
</div>

In [9]:
# import new locations of the tibia medial and lateral points into jupyter
# and recalculate the position of the tibial plateau center

lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

tibia_l_lat = lms_list[lms.GetControlPointIndexByLabel('tibia_l_lat')]
tibia_l_med = lms_list[lms.GetControlPointIndexByLabel('tibia_l_med')]

tibia_l_center = np.mean((tibia_l_lat, tibia_l_med), axis=0)

lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('tibia_l_center'), 
                               tibia_l_center[0], 
                               tibia_l_center[1], 
                               tibia_l_center[2])

In [10]:
# Reorient yellow slice to be strictly perpendicular to medio-lateral axis of the tibia 
# and center it at the center of the bone.
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
tibia_l_sagittal_plane_normal = (tibia_l_med - tibia_l_lat)/np.linalg.norm(tibia_l_med - tibia_l_lat)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, tibia_l_sagittal_plane_normal, tibia_l_center) 

<div class="alert alert-block alert-info">
 <b>Now that red, green and yellow slices are all aligned with the tibia axes, </b> place a number of landmarks that witll help a better description of the tibial shape. The toughest part is to describe the shape of the central and medial side of the tibial ephiphysis. Therefore, keep placement of the new markers strictly on the slices that coinside with the bone axes. 
    <p>on the yellow slice:  
    <li> <em>tibia_l_epiph_yellow_posteriormost</em> </li> 
    <li> <em>tibia_l_epiph_yellow_anterior_plate</em> </li> 
    <li> <em>tibia_l_epiph_yellow_patel_tend_point</em> </li> 
    <li> <em>tibia_l_ankle_yellow_anter</em> </li> 
    </p> 
    <p>on the green slice: 
    <li> <em>tibia_l_epiph_green_medial_lower_epiph</em> </li></p>

</div>

In [13]:
# return slices into the default positions
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_l_center)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_l_center)
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_l_center)

### Right tibia

In [14]:
# First, calculate tibial axes
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
ankle_r_center = lms_list[lms.GetControlPointIndexByLabel('ankle_r_center')]
tibia_r_normal = (knee_r_center-ankle_r_center)/np.linalg.norm(knee_r_center-ankle_r_center)

In [15]:
# Third, reorient red slice perpendicular to ankle-knee axis for the right tibia

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, knee_r_center)


<div class="alert alert-block alert-info">
    <b>In 3dSclicer GUI</b>, move the red slice to intersect with the uppermost part of the tibial plateau, and place the lateral and medial across the longest axis.
        </div>

In [16]:
# import medial and lateral points into jupyter
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

tibia_r_lat = lms_list[lms.GetControlPointIndexByLabel('tibia_r_lat')]
tibia_r_med = lms_list[lms.GetControlPointIndexByLabel('tibia_r_med')]



<div class="alert alert-block alert-info">
<b>While here</b>, adjust the position of landmarks on fibula: <em>fibula_r_as</em> is the anterior tip of the proximal end of the bone; <em>fibula_r_sup</em> - is the superior tip of the proximal head. These are best seen on the yellow slice but can be also controlled on the red slice.
</div>
<div class="alert alert-block alert-info">
    <b>Contunue using the same orientation of the red slice</b>, move the slice up so that it cuts across the widest part of the left patella, and adjust the lateral, medial, and anterior points of the patella in this view. The anterior point should be roughly in the middle of the frontal surface. Then, follow the code below to adjust the orientation of the yellow slice :
    </div>

In [17]:
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

patella_lat_r = lms_list[lms.GetControlPointIndexByLabel('patella_lat_r')]
patella_med_r = lms_list[lms.GetControlPointIndexByLabel('patella_med_r')]
patella_r_anter = lms_list[lms.GetControlPointIndexByLabel('patella_r_anter')]

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
patella_r_normal = (patella_med_r - patella_lat_r)/np.linalg.norm(patella_med_r - patella_lat_r)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, patella_r_normal, patella_r_anter)

<div class="alert alert-block alert-info">
 Then, <b>switch to the yellow slice</b> and display the anterior point of the patella on it. From here, move the slice so that the lowest tip of the patella is visible and adjust the position of <em>patella_r</em> landmark. Move <em>patella_sup_r</em> such that it demarcates the top of the left patella on the same yellow slice as <em>patella_r</em>.
</div>

In [18]:
# rotate the green slice so that it is parallel to patella mediolateral axis and center it at the ankle:
# the frontal plane should go through the ankle center and through the medial and lateral points on the tibial plateau
tibia_r_frontal_plane_normal = np.cross(tibia_r_normal, patella_r_normal)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, tibia_r_frontal_plane_normal, ankle_r_center)

<div class="alert alert-block alert-info">
 <b>As you look at the green slice, move the location of the red slice </b> so that it cuts across tha tibial epiphysis just below the joint surface. Adjust the position of <em>tibia_r_lat</em> and <em>tibia_r_med</em> so that they are visible on both, green and red slices, i.e to the plane that goes through the ankle centre.
</div>

In [19]:
# import new locations of the tibia medial and lateral points into jupyter
# and recalculate the position of the tibial plateau center

lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

tibia_r_lat = lms_list[lms.GetControlPointIndexByLabel('tibia_r_lat')]
tibia_r_med = lms_list[lms.GetControlPointIndexByLabel('tibia_r_med')]

tibia_r_center = np.mean((tibia_r_lat, tibia_r_med), axis=0)

lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('tibia_r_center'), 
                               tibia_r_center[0], 
                               tibia_r_center[1], 
                               tibia_r_center[2])

In [20]:
# Reorient yellow slice to be strictly perpendicular to medio-lateral axis of the tibia 
# and center it at the center of the bone.
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
tibia_r_sagittal_plane_normal = (tibia_r_med - tibia_r_lat)/np.linalg.norm(tibia_r_med - tibia_r_lat)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, tibia_r_sagittal_plane_normal, tibia_r_center) 

<div class="alert alert-block alert-info">
 <b>Now that red, green and yellow slices are all aligned with the tibia axes, </b> place a number of landmarks that witll help a better description of the tibial shape. The toughest part is to describe the shape of the central and medial side of the tibial ephiphysis. Therefore, keep placement of the new markers strictly on the slices that coinside with the bone axes. 
    <p>on the yellow slice:  
    <li> <em>tibia_r_epiph_yellow_posteriormost</em> </li> 
    <li> <em>tibia_r_epiph_yellow_anterior_plate</em> </li> 
    <li> <em>tibia_r_epiph_yellow_patel_tend_point</em> </li> 
    <li> <em>tibia_r_ankle_yellow_anter</em> </li> 
    </p> 
    <p>on the green slice: 
    <li> <em>tibia_r_epiph_green_medial_lower_epiph</em> </li></p>

</div>

In [21]:
# return slices into the default positions
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_r_center)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

## Locate midshaft points on tibias, fibulas and femora

In [10]:
## Left tibia
fibula_l_midshaft_anter = lms.GetControlPointIndexByLabel('fibula_l_midshaft_anter')
tibia_l_midshaft_anter = lms.GetControlPointIndexByLabel('tibia_l_midshaft_anter')
tibia_l_midshaft_poster = lms.GetControlPointIndexByLabel('tibia_l_midshaft_poster')
tibia_l_mean = np.mean((knee_l_center,ankle_l_center), axis=0)

In [29]:
### the next step will locate midshaft landmarks APPROXIMATELY in the middle of diaphisis
lms.SetNthControlPointPosition(fibula_l_midshaft_anter, 
                               lms_list[fibula_l_midshaft_anter][0], 
                               lms_list[fibula_l_midshaft_anter][1], 
                               tibia_l_mean[2])
lms.SetNthControlPointPosition(tibia_l_midshaft_anter, 
                               lms_list[tibia_l_midshaft_anter][0], 
                               lms_list[tibia_l_midshaft_anter][1], 
                               tibia_l_mean[2])
lms.SetNthControlPointPosition(tibia_l_midshaft_poster, 
                               lms_list[tibia_l_midshaft_poster][0], 
                               lms_list[tibia_l_midshaft_poster][1], 
                               tibia_l_mean[2])

In [11]:
### reorient the red slice:
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, tibia_l_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, adjust landmarks position to fit on the above slice exactly anterior or posterior of the shaft.
</div>

In [13]:
## Right tibia
fibula_r_midshaft_anter = lms.GetControlPointIndexByLabel('fibula_r_midshaft_anter')
tibia_r_midshaft_anter = lms.GetControlPointIndexByLabel('tibia_r_midshaft_anter')
tibia_r_midshaft_poster = lms.GetControlPointIndexByLabel('tibia_r_midshaft_poster')
tibia_r_mean = np.mean((knee_r_center,ankle_r_center), axis=0)

In [32]:
### the next step will locate midshaft landmars APPROXIMATELY in the middle of diaphisis
lms.SetNthControlPointPosition(fibula_r_midshaft_anter, 
                               lms_list[fibula_r_midshaft_anter][0], 
                               lms_list[fibula_r_midshaft_anter][1], 
                               tibia_r_mean[2])
lms.SetNthControlPointPosition(tibia_r_midshaft_anter, 
                               lms_list[tibia_r_midshaft_anter][0], 
                               lms_list[tibia_r_midshaft_anter][1], 
                               tibia_r_mean[2])
lms.SetNthControlPointPosition(tibia_r_midshaft_poster, 
                               lms_list[tibia_r_midshaft_poster][0], 
                               lms_list[tibia_r_midshaft_poster][1], 
                               tibia_r_mean[2])

In [14]:
### reorient the red slice:
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, tibia_r_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, adjust landmarks position to fit on this slice
</div>

In [34]:
## Left femur
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
femur_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_l_mean = np.mean((knee_l_center_coords, femur_l_center_coords), axis = 0)

femur_l_midshaft_anter = lms.GetControlPointIndexByLabel('femur_l_midshaft_anter')
lms.SetNthControlPointPosition(femur_l_midshaft_anter, 
                               lms_list[femur_l_midshaft_anter][0], 
                               lms_list[femur_l_midshaft_anter][1], 
                               femur_l_mean[2])

femur_l_midshaft_poster = lms.GetControlPointIndexByLabel('femur_l_midshaft_poster')
lms.SetNthControlPointPosition(femur_l_midshaft_poster, 
                               lms_list[femur_l_midshaft_poster][0], 
                               lms_list[femur_l_midshaft_poster][1], 
                               femur_l_mean[2])

In [ ]:
### reorient the red slice:
femur_l_normal = (femur_l_center_coords-knee_l_center_coords)/np.linalg.norm(femur_l_center_coords - knee_l_center_coords)
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_l_normal, femur_l_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust landmarks position to fit on the above slice.
</div>

In [35]:
## Right femur
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
femur_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_mean = np.mean((knee_r_center_coords, femur_r_center_coords), axis = 0)

femur_r_midshaft_anter = lms.GetControlPointIndexByLabel('femur_r_midshaft_anter')
lms.SetNthControlPointPosition(femur_r_midshaft_anter, 
                               lms_list[femur_r_midshaft_anter][0], 
                               lms_list[femur_r_midshaft_anter][1], 
                               femur_r_mean[2])

femur_r_midshaft_poster = lms.GetControlPointIndexByLabel('femur_r_midshaft_poster')
lms.SetNthControlPointPosition(femur_r_midshaft_poster, 
                               lms_list[femur_r_midshaft_poster][0], 
                               lms_list[femur_r_midshaft_poster][1], 
                               femur_r_mean[2])

In [36]:
### reorient the red slice:
femur_r_normal = (femur_r_center_coords-knee_r_center_coords)/np.linalg.norm(femur_r_center_coords - knee_r_center_coords)
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_r_normal, femur_r_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust landmarks position to fit on the above slice.
</div>

In [ ]:
### return red slice to normal orientation
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), femur_r_mean)

In [16]:
# Update location of joints in parent frames that were previousely omitted -- these should have been projected into the correct location anyway but just to make sure
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

pelvis_origin_in_ground_index = lms.GetControlPointIndexByLabel('pelvis_origin_in_ground')
pelvis_origin_coords = lms_list[lms.GetControlPointIndexByLabel('pelvis_origin')]
lms.SetNthControlPointPosition(pelvis_origin_in_ground_index, 
                               pelvis_origin_coords[0], 
                               pelvis_origin_coords[1], 
                               pelvis_origin_coords[2])

femur_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_l_center_in_pelvis_index = lms.GetControlPointIndexByLabel('femur_l_center_in_pelvis')
lms.SetNthControlPointPosition(femur_l_center_in_pelvis_index, 
                               femur_l_center_coords[0], 
                               femur_l_center_coords[1], 
                               femur_l_center_coords[2])

femur_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_center_in_pelvis_index = lms.GetControlPointIndexByLabel('femur_r_center_in_pelvis')
lms.SetNthControlPointPosition(femur_r_center_in_pelvis_index, 
                               femur_r_center_coords[0], 
                               femur_r_center_coords[1], 
                               femur_r_center_coords[2])